Choose the correct environment with all the dependencies install

In [1]:
#import all dependencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import pyvista as pv 
import pandas as pd
import trimesh
from scipy.spatial import KDTree
import os

Calculate high stress area

In [ ]:
#Load the metadata
cad_data = pd.read_csv('/Users/User/Documents/Project_JEB/JEB_Data/workingdata.csv')
cad_data['High_Stress_Area'] = 0  
#extract 'id' from the metadata set and access the corresponding volume and surface meshes
for part_id in cad_data['id']:
    
    print(part_id)
    
    vtk_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_volmesh_(vtk)/{part_id}.vtk"
    
    if part_id <= 308:
        fea_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_simresults_.csv._firsthalf/SimJEB_simresults_(csv)_firsthalf/{part_id}field.csv"
    else:
        fea_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_simresults_.csv._secondhalf/SimJEB_simresults_(csv)_secondhalf/{part_id}field.csv"
    
    #Load the volumetric mesh file
    mesh = pv.read(vtk_file_path)

    #Load the FEA data
    sim_data = pd.read_csv(fea_file_path)
    #select the stress type with maximum magnitude (vertical, horizontal, diagonal or torsional)
    stress_columns = ['ver_stress', 'hor_stress', 'dia_stress', 'tor_stress']
    sim_data['Stress'] = sim_data[stress_columns].max(axis=1)

    #add the stress values to the mesh points
    stress = sim_data['Stress'].values
    mesh.point_data['Stress'] = stress

    #select threshold value as required
    stress_threshold = 2500.0  
    critical_areas = mesh.threshold(value=stress_threshold, scalars='Stress')

    #add the total amount of area with stress above threshold to the metadata
    cad_data.loc[cad_data['id'] == part_id, 'High_Stress_Area'] = critical_areas.n_points


updated_cad_data_path = '/Users/User/Documents/Project_JEB/Results/Data_Analysis/updated_cad_data.csv'
cad_data.to_csv(updated_cad_data_path, index=False)


Calculate the critical stress area ratio

In [ ]:
#Load the metadata
cad_data = pd.read_csv('/Users/User/Documents/Project_JEB/JEB_Data/workingdata.csv')
cad_data['Critical_Area_Ratio'] = 0  
#extract 'id' from the metadata set and access the corresponding volume and surface meshes
for part_id in cad_data['id']:
    
    print(part_id)
    vtk_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_volmesh_(vtk)/{part_id}.vtk"
    if part_id <= 308:
        fea_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_simresults_.csv._firsthalf/SimJEB_simresults_(csv)_firsthalf/{part_id}field.csv"
    else:
        fea_file_path = f"/Users/User/Documents/Project_JEB/JEB_Data/SimJEB_simresults_.csv._secondhalf/SimJEB_simresults_(csv)_secondhalf/{part_id}field.csv"
    
    #Load the volumetric mesh file
    mesh = pv.read(vtk_file_path)

    #Load the FEA data
    sim_data = pd.read_csv(fea_file_path)
    #select the stress type with maximum magnitude (vertical, horizontal, diagonal or torsional)
    stress_columns = ['ver_stress', 'hor_stress', 'dia_stress', 'tor_stress']
    sim_data['Stress'] = sim_data[stress_columns].max(axis=1)

    #add the stress values to the mesh points
    stress = sim_data['Stress'].values
    mesh.point_data['Stress'] = stress

    #select threshold value as required
    stress_threshold = sim_data['Stress'].quantile(0.95)
    critical_areas = mesh.threshold(value=stress_threshold, scalars='Stress')

    #add the ratio of area with stress above threshold to the total surface area to the metadata
    cad_data.loc[cad_data['id'] == part_id, 'Critical_Area_Ratio'] = critical_areas.n_points / mesh.n_points



updated_cad_data_path = '/Users/User/Documents/Project_JEB/Results/Data_Analysis/updated_cad_data.csv'
cad_data.to_csv(updated_cad_data_path, index=False)
